<a href="https://colab.research.google.com/github/Sameeksha2597/AICourseProject/blob/main/02_train_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from datasets import load_from_disk
dataset = load_from_disk("/content/drive/MyDrive/AIproject/dataset")
print(dataset)

Mounted at /content/drive
DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})


In [ ]:
!pip install transformers datasets torch lime shap scikit-learn pandas numpy
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizerFast
from datasets import load_from_disk

# Load processed dataset
dataset = load_from_disk("/content/drive/MyDrive/AIproject/dataset")

# Model setup
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

# Select subsets for training
train_dataset_subset = dataset['train'].shuffle(seed=42).select(range(5000))  # subset for speed


# Training configuration
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/AIproject/results",
    per_device_train_batch_size=16,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_dir="/content/drive/MyDrive/AIproject/logs",
    report_to="none", # Disable wandb logging
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset_subset,
    eval_dataset={"validation": dataset['validation']},
)

trainer.train()

# Save model and tokenizer
model.save_pretrained("/content/drive/MyDrive/AIproject/models/distilbert_sst2")
tokenizer.save_pretrained("/content/drive/MyDrive/AIproject/models/distilbert_sst2")
!pip install nbformat

import nbformat

input_path = "03_explainability.ipynb"   # change per notebook
output_path = "03_explainability_clean.ipynb"

with open(input_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Remove widget metadata safely
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

with open(output_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print(f" Cleaned notebook saved as {output_path}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
